# Read German labels of occupations and devide them into masculine and feminine

In [1]:
import re
import json
import sys, glob, codecs
with codecs.open("de/Berufsbezeichnungen.txt", 'r','ISO 8859-16') as f:
    lines = [line.rstrip('\n').strip() for line in f]

masculine_beruf=[]
feminine_beruf=[]
neutral_beruf=[]
others_berufe=[]
skiped_berufe=[]
dipl_ing=[]
berufe={}
field=[]
fieldMarkers=["(weiterf" ,"(grundst","(T\xe4tigkeitsfeld)".decode('ISO 8859-16'),"(Staatse"]
replaceMarkers=[" (Uni)"," (FH)"," (Hochschule)"," (Textilveredlung)"," (Luftfahrt)"," (Uni/FH)",
                " (kirchl.Abschl./Mag.Theo.)"," -katholisch"," - alle Fachrichtungen"," -evangelisch",
                " - Freikirchen",
               "(FH/Uni)","(\xa766 BBiG/\xa742m HWO)".decode('ISO 8859-16')," (Versicherung)"," (HwO)","(EDV)",
               "(\xa766 BBiG/\xa742m HwO)".decode('ISO 8859-16'),"(\xa766BBiG/\xa742mHwO)".decode('ISO 8859-16')
                ,              
                "Orthodoxe/r","Ambulante/r","Technische/r","Dritte/r","Zweite/r","Erste/r","Bildende/r"]#work!!
maskuline_feminineMarkers={"steurer":"steurerin","steward":"stewardess","lotse":"lotsin","amter":"amtin",
                          "gehilfe":"gehilfin","arzt":"\xe4rztin".decode('ISO 8859-16'),"pfleger":"schwester",
                           "Modelleur":"Direktrice","Metallurge":"Metallurgin","beauftragter":"beauftragte",
                          "nwalt":"nw\xe4ltin".decode('ISO 8859-16'),"atrose":"atrosin",
                          "purser":"purserette","steuerer":"steuerin","iker":"ikerin",#analytiker#"ierer":"iererin", 
                            "koch":"k\xf6chin".decode('ISO 8859-16'),"aloge":"alogin","ologe":"ologin",
                        "p\xe4dagoge".decode('ISO 8859-16'):"p\xe4dagogin".decode('ISO 8859-16'),
                        "p\xe4dago".decode('ISO 8859-16'):"p\xe4dagogin".decode('ISO 8859-16'),
                        "P\xe4dagoge".decode('ISO 8859-16'):"P\xe4dagogin".decode('ISO 8859-16'),
                        "ogop\xe4de".decode('ISO 8859-16'):"ogop\xe4din".decode('ISO 8859-16'),
                           "mann":"frau","experte":"expertin","Script-man":"Script-girl" } 
feminine_maskulineMarkers={"stellte":"stellter","beauftragte":"beauftragter","schwester":"pfleger",
                           "mutter":"vater"}

def check_dot(new_beruf):

    if "Freiwil." in new_beruf:
        new_beruf=new_beruf.replace("Freiwil.","Freiwillige/r")
    elif "Funktionsdiagn." in new_beruf:
        new_beruf=new_beruf.replace("Funktionsdiagn.","Funktionsdiagnostik")
    elif "Laboratoriumst." in new_beruf:
        new_beruf=new_beruf.replace("Laboratoriumst.","Laboratoriumstechnik")
    elif "Ausr\xfcstungst.".decode('ISO 8859-16')in new_beruf:
        new_beruf=new_beruf.replace("Ausr\xfcstungst.".decode('ISO 8859-16'),"Ausr\xfcstungstechnik".decode('ISO 8859-16'))
    elif "Landwirt." in new_beruf:
        new_beruf=new_beruf.replace("Landwirt.","Landwirtschaftliche")
    elif "Knappsch." in new_beruf:
        new_beruf=new_beruf.replace("Knappsch.","Knappschaftliche")
    elif "Allg." in new_beruf:
        new_beruf=new_beruf.replace("Allg.","Allgemeine") 
    elif "-ev." in new_beruf:
        new_beruf=new_beruf.replace("-ev.","-evangelische")
    elif "Techn." in new_beruf:
        new_beruf=new_beruf.replace("Techn.","Technische/r")
    elif "Krankenk." in new_beruf:
        new_beruf=new_beruf.replace("Krankenk.","Krankenkassen")
    elif "Schiffsoff." in new_beruf:
        new_beruf=new_beruf.replace("Schiffsoff.","Schiffsoffizier/in")
    elif "-kath." in new_beruf:
        new_beruf=new_beruf.replace("-kath.","-katholische")  
    elif "Abfallwirt." in new_beruf:
        new_beruf=new_beruf.replace("Abfallwirt.","Abfallwirtschaft") ####
    elif "Med. Doku." in new_beruf:
        new_beruf=new_beruf.replace("Med. Doku.","Medizinische Dokumentation")####
    elif "Wasserinst." in new_beruf:
        new_beruf=new_beruf.replace("Wasserinst.","Wasserinstallation")###
    elif "studiotechn. Anlag." in new_beruf:
        new_beruf=new_beruf.replace("studiotechn. Anlag.","studiotechnischen Anlagen")###
    elif "romasch." in new_beruf:
        new_beruf=new_beruf.replace("romasch.","romaschinen")###Büromaschinen
    elif "Dienstleist." in new_beruf:
        new_beruf=new_beruf.replace("Dienstleist.","Dienstleistungen")
    elif "techn." in new_beruf:
        new_beruf=new_beruf.replace("techn.","technisches")
    elif "Baumaschinent." in new_beruf:
        new_beruf=new_beruf.replace("Baumaschinent.","Baumaschinentechnik")     
    if "Sozialvers." in new_beruf:
        new_beruf=new_beruf.replace("Sozialvers.","Sozialversicherung")
    if "Krankenversich." in new_beruf:
        new_beruf=new_beruf.replace("Krankenversich.","Krankenversicherung")
    if "verwalt." in new_beruf:
        new_beruf=new_beruf.replace("verwalt.","verwaltung")
    if "Info. und Doku" in new_beruf:
        new_beruf=new_beruf.replace("Info. und Doku","Information und Dokumentation")
    if "Info." in new_beruf:
        new_beruf=new_beruf.replace("Info.Dienste","Informationsdienste")
    if "Schmuckwarenind." in new_beruf:
        new_beruf=new_beruf.replace("Schmuckwarenind.","Schmuckwarenindustrie") 
    if " u. " in new_beruf:
        new_beruf=new_beruf.replace(" u. "," und ") 
    if " med." in new_beruf:
        new_beruf=new_beruf.replace(" med."," medizinische") 
    if "Nahrungsmittelhandw.(" in new_beruf:
        new_beruf=new_beruf.replace("Nahrungsmittelhandw.(","Nahrungsmittelhandwerk (")
    return new_beruf
berufe_with_3=[]
all_with_fur=[]
all_with_fur_neutral=[]
all_with_fur_berufe={}
for beruf in lines:
    if "Schmuckwarenind." in beruf:
            beruf=beruf.replace("Schmuckwarenind.","Schmuckwarenindustrie") 
    if ("/r" in beruf)&(" f\xfcr ".decode('ISO 8859-16')not in beruf):
        if  "(er/in)" in beruf:
            pass
            #nothing is here
        elif "(e/in)" in beruf:
            pass
            #nothing is here
        elif "(e/r)" in beruf:
            if "/in" in beruf:
                #"Evangelisch(e/r) Pfarrer/in"
                berufe[beruf.replace("(e/r)","er").replace("/in","")]=beruf.replace("(e/r)","e").replace("/in","in")
            else:
                #"Evangelisch(e/r) Theologe/Theologin (Uni)"
                new_beruf=beruf.replace("(e/r)","-er")
                new_beruf=re.sub("[*?\(\[].*?[\)\]]", "", new_beruf).strip()
                new_beruf=new_beruf.split('/')
                berufe[new_beruf[0].replace("-er","er")]=new_beruf[0].replace("-er","e").split(" ")[0]+" "+new_beruf[1]            
        elif "(r)" in beruf:
            new_beruf=beruf.replace("(r)","/r")
            new_beruf=re.sub("[*?\(\[].*?[\)\]]", "", new_beruf).strip()
            berufe[new_beruf.replace("/r","r").replace("/in","")]=new_beruf.replace("/r","").replace("/in","in")
        else:
            new_beruf=re.sub("[*?\(\[].*?[\)\]]", "", beruf).strip()  
            new_beruf=new_beruf.replace(" u."," und").replace(".u.",". und ").strip()
            if "." in new_beruf:
                new_beruf=check_dot(new_beruf) 
            if " Gesundheitswesen" in new_beruf:
                new_beruf=new_beruf.replace(" Gesundheitswesen", " - Gesundheitswesen")
            new_beruf_=new_beruf.split(" - ")    
            if len(new_beruf_)>=2:
                if ("/" in new_beruf_[1])&("/r" not in new_beruf_[1]):                 
                    #print "==",new_beruf
                    if "-/" in new_beruf_[1]:
                        if new_beruf_[1].count("-/")==1:
                            new_beruf=new_beruf.replace("-/", "- und ")
                        else:
                            new_beruf=new_beruf.replace("-/", "-, ")
                    elif new_beruf_[1].count("/")==1:                        
                        new_beruf=new_beruf_[0]+" - "+new_beruf_[1].replace("/", " und ")
                    else:
                        new_beruf=new_beruf_[0]+" - "+new_beruf_[1].replace("/", ", ")
            if " / " in new_beruf:
                #Sozialpädagogischer Assistent / Kinderpfleger
                new_beruf=new_beruf.split(' / ')[0]
                #here we cut second word but we should check if it is exist(doesnt done)
            new_beruf_=new_beruf.split('/')
            if len(new_beruf_)==2:
                new_beruf_m=new_beruf.replace("/r","r").replace("/in","")
                new_beruf_f=new_beruf.replace("/r","").replace("/in","in")
                #Buchprüfer/in (vereidigte/r)
                berufe[new_beruf_m]=new_beruf_f
            elif len(new_beruf_)>2:                
                if "/in" in new_beruf:
                    new_beruf_m=new_beruf.replace("/r","r").replace("/in","")
                    new_beruf_f=new_beruf.replace("/r","").replace("/in","in")
                    if "/Kapit\xe4n".decode('ISO 8859-16') in new_beruf_m:
                        new_beruf_m=new_beruf_m.replace("/Kapit\xe4n".decode('ISO 8859-16'),"")
                        new_beruf_f=new_beruf_f.replace("/Kapit\xe4nin".decode('ISO 8859-16'),"")
                        berufe["Kapit\xe4n".decode('ISO 8859-16')]="Kapit\xe4nin".decode('ISO 8859-16')
                    berufe[new_beruf_m]=new_beruf_f
                elif "/-frau" in new_beruf:
                    new_beruf_m=new_beruf.replace("/r","r").replace("/-frau","")
                    new_beruf_f=new_beruf.replace("/r","").replace("mann/-frau","frau")
                    berufe[new_beruf_m]=new_beruf_f
                else:
                    if (new_beruf_[1][:1]=="r")&(new_beruf_[2][:1]=="r"):
                        new_beruf_m=new_beruf.replace("/r","r")
                        new_beruf_f=new_beruf.replace("/r","")
                        berufe[new_beruf_m]=new_beruf_f
                    elif ("-/" not in new_beruf)&("/-"  not in new_beruf):
                        new_beruf_m=new_beruf_[0]+new_beruf_[1]
                        new_beruf_f=new_beruf_[0]+" "+new_beruf_[2]
                        berufe[new_beruf_m]=new_beruf_f                      
                    else:
                        new_beruf_m=new_beruf.replace("/r","r")
                        new_beruf_f=new_beruf.replace("/r","")
                        if ("-/" in new_beruf_m):
                            new_beruf_m=new_beruf_m.replace("-/", "- und ")
                            new_beruf_f=new_beruf_f.replace("-/", "- und ")
                            berufe[new_beruf_m]=new_beruf_f
                        else:
                            #"Technische/r Aufsichtsbeamter/-beamtin"
                            new_beruf_m=new_beruf_m.replace("/-beamtin","")
                            new_beruf_f=new_beruf_f.replace("beamter/-beamtin", "beamtin") 
                            berufe[new_beruf_m]=new_beruf_f                          
            elif len(new_beruf_)==1:
                #nothin is here
                print "0000000",new_beruf
            if " - " in new_beruf_m:
                new_beruf_=new_beruf_m.split(" - ")
                #e.g. Technischer Assistent - Meteorologie ===> try ohne " - Meteorologie"
                if new_beruf_[0] not  in berufe.keys():
                    berufe[new_beruf_[0]]=new_beruf_f.split(" - ")[0]  
            
            #Betriebswirtschaftlicher Assistent Gesundheitswesen 
    else:
        for r in replaceMarkers:
            beruf=beruf.replace(r, "")
        #before here was deleting of für
        #start checking
        if "(er/in)" in beruf:
            new_beruf=beruf.split('(er/in)')
            masculine_beruf.append(new_beruf[0].strip()+"er")
            feminine_beruf.append(new_beruf[0].strip()+"in")
            berufe[new_beruf[0].strip()+"er"]=new_beruf[0].strip()+"in"
        elif "(e/in)" in beruf:
            new_beruf=beruf.split('(e/in)')
            if "/" not in new_beruf[0]:
                masculine_beruf.append(new_beruf[0].strip()+"e")
                feminine_beruf.append(new_beruf[0].strip()+"in")
                berufe[new_beruf[0]+"e"]=new_beruf[0].strip()+"in"  
            else:
                others_berufe.append(new_beruf)
        elif "(e/r)" in beruf:
            #we will work with this words in script "al_with_r", so nothing is here
            pass
        elif "(r)" in beruf:            
            if " "in beruf:
                new_beruf=beruf.split('/')
                if new_beruf[1][:2]=="in":
                    berufe[beruf.replace("(r)","r").replace("/in","")]=beruf.replace("(r)","").replace("/in","in")
                else:
                    print beruf
                    #nothing is here
            else:
                #Blumenbinde(r)meister/in
                berufe[beruf.replace("(r)","r").replace("/in","")]=beruf.replace("(r)","").replace("/in","in")           
        elif any([x in beruf for x in fieldMarkers]):
            temp=re.sub("[*?\(\[].*?[\)\]]", "", beruf).strip()#delete everything in brackets
            field.append(temp)
        else:    
            if " f\xfcr ".decode('ISO 8859-16') in beruf: #95 items
                beruf=beruf.replace(" u."," und").replace(".u.",". und ").strip()
                #all_with_fur.append(beruf)
                beruf_=beruf.split(" f\xfcr ".decode('ISO 8859-16'))
                b_with_fur=beruf_[0].split('/')
                if len(b_with_fur)==1:
                    if "Dipl." in beruf:
                        dipl_ing.append(new_beruf)
                    else:
                        all_with_fur_neutral.append(beruf)
                elif len(b_with_fur)==2:
                    all_with_fur.append(beruf)
                else:
                    #print b_with_fur
                    #[u'Fachverk\xe4ufer', u'-berater', u'in']
                    #[u'Techniker', u'in - Fahrlehrer', u'in']
                    if (b_with_fur[1][:2]=="in")&(b_with_fur[2][:2]=="in"):
                        all_with_fur_m=b_with_fur[0]+b_with_fur[1][2:]+" f\xfcr ".decode('ISO 8859-16')+beruf_[1]
                        all_with_fur_f=b_with_fur[0]+b_with_fur[1]+b_with_fur[2]+" f\xfcr ".decode('ISO 8859-16')+beruf_[1]
                    else:
                        all_with_fur_m=b_with_fur[0]+", "+b_with_fur[1]+" f\xfcr ".decode('ISO 8859-16')+beruf_[1]
                        all_with_fur_f=b_with_fur[0]+"in, "+b_with_fur[1]+b_with_fur[2]+" f\xfcr ".decode('ISO 8859-16')+beruf_[1]
                    if "/" in all_with_fur_m:
                        all_with_fur_m=all_with_fur_m.replace("/"," und ")
                        all_with_fur_f=all_with_fur_f.replace("/"," und ")
                    all_with_fur_berufe[all_with_fur_m]=all_with_fur_f
                beruf=beruf.split(" f\xfcr ".decode('ISO 8859-16'))[0].strip()
            beruf_=re.sub("[*?\(\[].*?[\)\]]", "", beruf).strip()#delete everything in brackets
            #beruf_=check_dot(beruf_)
            new_beruf=beruf_.split('/')
            if (len(new_beruf)==2) | (len(new_beruf)==3):
                if len(new_beruf)==3:
                    berufe_with_3.append(beruf_)
                new_beruf[0]=new_beruf[0].strip()
                new_beruf[1]=new_beruf[1].strip()
                if new_beruf[1][:3]=="in)":
                    if new_beruf[0][-1:]=="e":
                        masculine_beruf.append(new_beruf[0].replace("(", ""))
                        feminine_beruf.append(new_beruf[0].replace("(e", "")+"in")
                        berufe[new_beruf[0].replace("(", "")]=new_beruf[0].replace("(e", "")+"in"
                    else:    
                        masculine_beruf.append(new_beruf[0].replace("(", ""))
                        feminine_beruf.append(new_beruf[0][:-3]+"in")
                        berufe[new_beruf[0].replace("(", "")]=new_beruf[0][:-3]+"in"
                elif new_beruf[1][:2]=="in":
                    if new_beruf[0][-1:]=="e":
                        masculine_beruf.append(new_beruf[0])
                        feminine_beruf.append(new_beruf[0][:-1]+"in")
                        berufe[new_beruf[0]]=new_beruf[0][:-1]+"in"
                    else:
                        masculine_beruf.append(new_beruf[0])
                        feminine_beruf.append(new_beruf[0]+"in")
                        if "Vorpolierer" in new_beruf[0]:
                            berufe[new_beruf[0]+new_beruf[1].replace("in -","")]=new_beruf[0]+new_beruf[1].replace("in -","in")
                        elif "Industriekeramiker" in new_beruf[0]:
                            berufe[new_beruf[0]+new_beruf[1].replace("in -","").replace("in "," ")]=new_beruf[0]+new_beruf[1].replace("in -","in")
                        elif new_beruf[0]=="Textilgestalter":
                            berufe[new_beruf[0]+" im Handwerk"]=new_beruf[0]+"in im Handwerk"
                        elif ("Produktgestalter" in new_beruf[0])|("Produktpr" in new_beruf[0])|("Produktveredler" in new_beruf[0]):#Produktprüfer
                            berufe[new_beruf[0]+new_beruf[1].replace("in -","")]=new_beruf[0]+new_beruf[1].replace(" -","")
                        elif("Betonstahlbieger" in new_beruf[0])&(len(new_beruf)==3):
                            berufe["Betonstahlbieger und -flechter"]="Betonstahlbiegerin und -flechterin"
                        elif("Fachberater" in new_beruf[0]):
                            berufe[new_beruf[0]+new_beruf[1][2:]]=new_beruf[0]+new_beruf[1]
                        elif ("Fachverk\xe4ufer".decode('ISO 8859-16') in new_beruf[0]):
                            m_beruf=beruf.replace("/in","").strip()
                            f_beruf=beruf.replace("/in","in").strip()
                            if "/" in m_beruf:
                                m_beruf=m_beruf.replace("/"," und ")
                                f_beruf=f_beruf.replace("/"," und ")
                            if "." in m_beruf:
                                m_beruf=check_dot(m_beruf)
                                f_beruf=check_dot(f_beruf)
                            berufe[m_beruf]=f_beruf
                        elif "Brauer/in und M" in beruf_:
                            berufe[beruf_.replace("/in","")]=beruf_.replace("/in","in")
                        else:
                            berufe[new_beruf[0]]=new_beruf[0]+"in"
                elif new_beruf[1]=="i":
                    masculine_beruf.append(new_beruf[0])
                    feminine_beruf.append(new_beruf[0]+"i")
                    berufe[new_beruf[0]]=new_beruf[0]+"i"
                elif new_beruf[1]=="r":
                    masculine_beruf.append(new_beruf[0]+"r")
                    feminine_beruf.append(new_beruf[0])
                    berufe[new_beruf[0]+"r"]=new_beruf[0]

                elif any([new_beruf[0][-len(x):]==x for x in maskuline_feminineMarkers]): 
                    #print new_beruf
                    for r in maskuline_feminineMarkers:
                        if "-gehilfin-" in new_beruf[1]:
                            if "-techn." in new_beruf[0]:
                                new_beruf[0]=new_beruf[0].replace("-techn.","-technische/r") 
                            if "-techn." in new_beruf[1]:
                                new_beruf[1]=new_beruf[1].replace("-techn.","-technische/r")
                            new_beruf_m=new_beruf[0]+new_beruf[1].replace("-gehilfin-"," ")
                            new_beruf_f=new_beruf[0].replace("gehilfe","")+new_beruf[1].replace("-gehilfin-","gehilfin ")
                            berufe[new_beruf_m]=new_beruf_f
                        elif "Servicekaufmann" in new_beruf[0]:
                            berufe[new_beruf[0]+new_beruf[1].replace("-frau","")]=new_beruf[0].replace(r,maskuline_feminineMarkers[r])+new_beruf[1].replace("-frau","")
                        elif "Medientechnologe" in new_beruf[0]:
                            berufe[new_beruf[0]+new_beruf[1].replace("-technologin","")]=new_beruf[0][:-1]+"in"+new_beruf[1].replace("-technologin","")
                        elif new_beruf[0][-len(r):]==r:#if r in new_beruf[0]:
                            masculine_beruf.append(new_beruf[0])
                            feminine_beruf.append(new_beruf[0].replace(r,maskuline_feminineMarkers[r]))
                            berufe[new_beruf[0]]=new_beruf[0].replace(r,maskuline_feminineMarkers[r])
                    
                elif any([new_beruf[0][-len(x):]==x for x in feminine_maskulineMarkers]):
                    #print new_beruf
                    for r in feminine_maskulineMarkers:
                        if new_beruf[0][-len(r):]==r:#if r in new_beruf[0]:
                            feminine_beruf.append(new_beruf[0])
                            masculine_beruf.append(new_beruf[0].replace(r,feminine_maskulineMarkers[r]))
                            berufe[new_beruf[0].replace(r,feminine_maskulineMarkers[r])]=new_beruf[0]

                elif ("-"not in new_beruf[0])&("-"not in new_beruf[1])&(new_beruf[0][0].isupper())&(new_beruf[1][0].isupper()): 
                    if (" "not in new_beruf[0])&(" " not in new_beruf[1]):
                        #print new_beruf
                        masculine_beruf.append(new_beruf[0])
                        feminine_beruf.append(new_beruf[1])
                        if new_beruf[1]=="Personalvermittler":
                             berufe[new_beruf[1]]= new_beruf[1]+"in"
                        else:
                            berufe[new_beruf[0]]=new_beruf[1]
                    elif (new_beruf[0].split(" ")[0] in masculine_beruf) & (new_beruf[1].split(" ")[0] in feminine_beruf):
                        pass #print "We already HAVE it"
                    else: 
                        skiped_berufe.append(new_beruf) 
                        #!!!!!then we will work with them manually
                elif ("Dipl.-Ing. " in new_beruf[0] ) | ("Dipl.Ing. "in new_beruf[0]):
                    dipl_ing.append(new_beruf)
                    #neutral_beruf.append("Dipl.-Ing.") #should we add other variations?
                elif "Verfahrensmech." in new_beruf[0]:
                    pass
                else:        
                    if (new_beruf[0].split(" ")[0] in masculine_beruf) & (new_beruf[1].split(" ")[0] in feminine_beruf) :
                        pass #print "We already HAVE it"
                    else:
                        if "kraft" in new_beruf[0] :
                            neutral_beruf.append(new_beruf[0])
                        else:
                            skiped_berufe.append(new_beruf)
                            #!!!!!then we will work with them manually #file "other.json"
            elif len(new_beruf)>3: 
                new_beruf[0]=new_beruf[0].strip()
                new_beruf[1]=new_beruf[1].strip()
                new_beruf[2]=new_beruf[2].strip()
                new_beruf[3]=new_beruf[3].strip()

                if new_beruf[0].strip() not in masculine_beruf:
                    if (len(new_beruf)==4 ) & (new_beruf[1].strip()=="in")& (new_beruf[3].strip()=="in")& (new_beruf[2].strip()[0].isupper()):
                        masculine_beruf.append(new_beruf[0])
                        masculine_beruf.append(new_beruf[2])
                        feminine_beruf.append(new_beruf[0]+"in")
                        feminine_beruf.append(new_beruf[2]+"in")
                        berufe[new_beruf[0]]=new_beruf[0]+"in"
                        berufe[new_beruf[2]]=new_beruf[2]+"in"
                    elif (len(new_beruf)==4 ) & (new_beruf[1].strip()=="in")& (new_beruf[3][:5]=="-frau")& (new_beruf[2].strip()[0].isupper()):
                        masculine_beruf.append(new_beruf[0])
                        masculine_beruf.append(new_beruf[2])
                        feminine_beruf.append(new_beruf[0]+"in")
                        feminine_beruf.append(new_beruf[2][:-4]+"frau")
                        berufe[new_beruf[0]]=new_beruf[0]+"in"
                        berufe[new_beruf[2]]=new_beruf[2][:-4]+"frau"
                    else:
                        if ("Dipl.-Ing. " in new_beruf[0]) | ("Dipl.Ing. "in new_beruf[0]):
                            dipl_ing.append(new_beruf)
                        else:
                            others_berufe.append(new_beruf)
            elif len(new_beruf)==1:
                if ("Dipl.-Ing. " in new_beruf[0]) | ("Dipl.Ing. "in new_beruf[0]):
                    dipl_ing.append(new_beruf)
                else:
                    if new_beruf[0] not  in masculine_beruf:
                        neutral_beruf.append(new_beruf[0])
masculine_beruf=list(set(masculine_beruf))
feminine_beruf=list(set(feminine_beruf)) 
neutral_beruf=list(set(neutral_beruf))

for i in berufe:
    berufe[i]=berufe[i].strip()
         
    
with open('de/all_with_fur.json', 'w') as f:
    json.dump(all_with_fur,  f, encoding='ISO 8859-16', indent=4)
with open('de/all_with_fur_neutral.json', 'w') as f:
    json.dump(all_with_fur_neutral,  f, encoding='ISO 8859-16', indent=4)
with open('de/all_with_fur_berufe.json', 'w') as f:
    json.dump(all_with_fur_berufe,  f, encoding='ISO 8859-16', indent=4)

with open('de/berufe_with_3.json', 'w') as f:
    json.dump(berufe_with_3,  f, encoding='ISO 8859-16', indent=4)



In [2]:
with codecs.open("de/other_manually_retrieved.json", 'r','ISO 8859-16') as f:
    other_cleaned = json.load(f)
nn=[]
for j in other_cleaned:
    if (len(j)==2) & (j[0][0].isupper())&(j[1][0].isupper() ):
        berufe[j[0]]=j[1]
    elif (len(j)==2) &(j[1]=="in"):
        berufe[j[0]]=j[0]+"in"        
    elif (len(j)==4) & (j[1]=="in") & (j[3]=="in") & (j[0][0].isupper()) & (j[2][0].isupper()):
        berufe[j[0]]=j[0]+"in" 
        berufe[j[2]]=j[2]+"in" 
    elif (len(j)==4) &(j[0][0].isupper()) & (j[1][0].isupper()) & (j[2][0].isupper()) & (j[3][0].isupper()):
        berufe[j[0]]=j[1] 
        berufe[j[2]]=j[3] 
    else:
        nn.append(j)
        
print len(other_cleaned)        
print len(nn)    

with codecs.open("de/field_manually_retrieved.json", 'r','ISO 8859-16') as f:
    field_other = json.load(f)
field.extend(field_other)  

57
0


In [3]:
with open('de/masculine.json', 'w') as f:
    json.dump(masculine_beruf,  f, encoding='ISO 8859-16', indent=4)
with open('de/feminine.json', 'w') as f:
    json.dump(feminine_beruf,  f, encoding='ISO 8859-16', indent=4)
with open('de/neutral.json', 'w') as f:
    json.dump(neutral_beruf,  f, encoding='ISO 8859-16', indent=4)    
with open('de/dipl_ing.json', 'w') as f:
    json.dump(dipl_ing,  f, encoding='ISO 8859-16', indent=4)

all_other=[]
all_other=others_berufe
all_other.extend( skiped_berufe )


with open('de/other.json', 'w') as f:
    json.dump(all_other,  f, encoding='ISO 8859-16', indent=4)

    
if "Hebamme" in berufe.keys():
    berufe["Entbindungspfleger"] = "Hebamme"
    del berufe["Hebamme"]
if berufe.has_key("Hostess"):
    berufe["Host"] = "Hostess"
    del berufe["Hostess"] 
if berufe.has_key("Fremdsprachensteward"):
    berufe["Fremdsprachensteward"] = "Fremdsprachenhostess" 
    del berufe["Fremdsprachensteward"]
if berufe.has_key("Hausdame"):
    berufe["Housekeeper"] = "Hausdame" 
    del berufe["Hausdame"] 
if berufe.has_key("Mannequin"):
    berufe["Dressman"] = "Mannequin" 
    del berufe["Mannequin"] 
if berufe.has_key("Krankenschwester"):
    del berufe["Krankenschwester"]
    print "found Krankenschwester"
    
to_del=[]
for b_m in berufe: 
    if (b_m[len(b_m)-1]=="e")&(berufe[b_m][len(berufe[b_m])-2:len(berufe[b_m])]=="er"):
        to_del.append(b_m)
if len(to_del)>1:
    for b_m in to_del:
        berufe[berufe[b_m]] = b_m
        del berufe[b_m]
elif len(to_del)==1:
    berufe[berufe[to_del[0]]] = to_del[0]
    del berufe[to_del[0]]
    
with open('de/berufe.json', 'w') as f:
    json.dump(berufe,  f, encoding='ISO 8859-16', indent=4)    
with open('de/field.json', 'w') as f:
    json.dump(field,  f, encoding='ISO 8859-16', indent=4)
    
print len(all_other)
#Metallschleifer/in und -polierer/in

80


In [4]:
def store_only_simple_berufe(beruf_list,name_store):
    j=0
    simple=[]
    difficult=[]
    for i in beruf_list:
        if ("- u" in i) | ("-, " in i):
            j+=1
            simple.append(i.rsplit(' ', 1)[1])#return only last world after white space
        elif " " in i:
            difficult.append(i)
        else:
            simple.append(i)
    with open('de/simple_{0}.json'.format(name_store), 'w') as f:
        json.dump(simple,  f, encoding='ISO 8859-16', indent=4)
        
        
store_only_simple_berufe(feminine_beruf,"feminine")
store_only_simple_berufe(masculine_beruf,"masculine")



# Check for mistakes and fix them

In [5]:
fremd_words=["Avid","Finno","Corporate","General", "Media","Account","Media","Management","Pre-Opening" ]
i=0
add_new={}
del_old=[]
for beruf in berufe:
    if ("- u" in beruf) | ("-, " in beruf)| (". " in beruf)| (" - " in beruf)| ("- " in beruf) | (", " in beruf):
        if (", " in beruf):
            #print beruf
            #change the feminine and masculine form
            pass
        else:
            pass
        i+=1
    elif " " in beruf:
        
        splitted=beruf.split(" ",1)
        if ("Werkgehilfe " in beruf)|("Industriekeramiker" in beruf)|("Produktgestalter" in beruf)|("Produktveredler" in beruf):
            pass
        elif ("Textilgestalter" in beruf)|("Medientechnologe" in beruf)|("Produktpr" in beruf)|("Betonstahlbieger" in beruf):
            pass
        elif ("Fachverk\xe4ufer".decode('ISO 8859-16') in beruf)|("Brauer und M"in beruf):
            pass
        elif splitted[0] not in fremd_words:
            if (splitted[0]!="Masseur") & (splitted[0]!="Polsterer")& (splitted[0]!="Verantwortlicher"):
                #change the feminine and masculine form
                
                if splitted[0][-1:]!="r":
                    #change last letter
                    #add new and del old beruf
                    if splitted[0][-1:]=="e":
                        name= splitted[0]+"r "+splitted[1]
                        add_new[name]=splitted[0]+" "+berufe[beruf].split(" ",1)[1]
                        print beruf
                        print "==we will add r in masculine"
                    else:
                        name= splitted[0]+"er "+splitted[1]
                        add_new[name]=splitted[0]+"e "+berufe[beruf].split(" ",1)[1]
                        print beruf
                        print "==we will add er in masculine and e in feminine"
                    del_old.append(beruf)
                elif (berufe[beruf][:len(splitted[0])]==splitted[0]):#if first word same as masculine
                    #del letter "r" in first word
                    add_new[beruf]=splitted[0][:-1]+" "+berufe[beruf].split(" ",1)[1]
                    print beruf
                    print "==we will del r in feminine"
            elif splitted[0]=="Masseur":
                pass
            elif splitted[0]!="Polsterer":
                pass
for b in add_new:
    berufe[b]=add_new[b]

if len(del_old)>1:
    for b in del_old:
        del berufe[b]
elif len(del_old)==1:
    del berufe[del_old[0]]
            
print i   

with open('de/berufe.json', 'w') as f:
    json.dump(berufe,  f, encoding='ISO 8859-16', indent=4)  

Katholischer Theologe
==we will del r in feminine
Fremdsprachlicher Wirtschaftskorrespondent
==we will del r in feminine
356


# add professions without "Dipl*" substring and del all with "Dipl"

In [6]:
import json

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
berufe=load_simple_json('de/berufe.json')  



def upcase_first_letter(s):
    return s[0].upper() + s[1:]

i=0
j=0
to_del=[]
berufe_add={}
for beruf in berufe:
    if "Dipl" in beruf:
        #print beruf
        i+=1
        to_del.append(beruf)
    if "ing." in beruf:
        if "Dipl.-" in beruf:
            new_beruf=beruf.replace("Dipl.-","").replace("ing.","ingenieur")
            new_beruf_f=beruf.replace("ing.in","ingenieurin").replace("ing.","ingenieurin").replace("Dipl.-","")
            if not berufe.has_key(new_beruf):
                berufe_add[new_beruf]=new_beruf_f
        else:
            new_beruf=beruf.replace("ing.","ingenieur")
            new_beruf_f=beruf.replace("ing.in","ingenieurin").replace("ing.","ingenieurin")
            if not berufe.has_key(new_beruf):
                berufe_add[new_beruf]=new_beruf_f
    elif "Dipl.-" in beruf: #Ev. Dipl.-Religionspädagoge #Kath. Dipl.- Religionspädagoge
        new_beruf=beruf.split('Dipl.-')[1].strip()
        new_beruf_f=berufe[beruf].split('Dipl.-')[1].strip()
        if berufe.has_key(new_beruf):
            #print "we have ",new_beruf
            pass
        else:
            print "!!! new !! ",new_beruf
            j+=1
            berufe_add[new_beruf]=new_beruf_f
    elif "Diplom-" in beruf:
        new_beruf=beruf.split("Diplom-")[1].strip()
        new_beruf_f=berufe[beruf].split("Diplom-")[1].strip()
        if berufe.has_key(new_beruf):
            #print "we have ",new_beruf
            pass
        else:
            print "!!! new !! ",new_beruf
            j+=1
            berufe_add[new_beruf]=new_beruf_f
    elif "Diplom" in beruf:
        #make first letter big and check this word
        new_beruf=upcase_first_letter(beruf.split("Diplom")[1].strip())
        new_beruf_f=upcase_first_letter(berufe[beruf].split("Diplom")[1].strip())
        if berufe.has_key(new_beruf):
            #print "we have ",new_beruf
            pass
        else:
            print "!!! new !! ",new_beruf
            j+=1
            berufe_add[new_beruf]=new_beruf_f
    elif "Dipl. " in beruf:
        new_beruf=beruf.split("Dipl. ")[1].strip()
        new_beruf_f=berufe[beruf].split("Dipl. ")[1].strip()
        if berufe.has_key(new_beruf):
            #print "we have ",new_beruf
            pass
        else:
            print "!!! new !! ",new_beruf
            j+=1
            berufe_add[new_beruf]=new_beruf_f
    elif "Dipl." in beruf:
        new_beruf=beruf.split("Dipl.")[1].strip()
        new_beruf_f=berufe[beruf].split("Dipl.")[1].strip()
        if berufe.has_key(new_beruf):
            #print "we have ",new_beruf
            pass
        else:
            print "!!! new !! ",new_beruf
            j+=1
            berufe_add[new_beruf]=new_beruf_f
    
print i
print j

#del old version
for i_i in to_del:
    del berufe[i_i]

for b in berufe_add:
    berufe[b]=berufe_add[b]
with open('de/berufe.json', 'w') as f:
    json.dump(berufe,  f, encoding='ISO 8859-16', indent=4)  

!!! new !!  Patholinguist
!!! new !!  Verwaltungsökonom
!!! new !!  Orientarchäologe
!!! new !!  Nordist
!!! new !!  Filmwirtschaftler
!!! new !!  Wissenschaftsorganisator
!!! new !!  Lateinamerikawissenschaftler
!!! new !!  Praehistoriker
!!! new !!  Design-Ingenieur
!!! new !!  Landschaftsökologe
!!! new !!  Sozialjurist
!!! new !!  Hungarologe
!!! new !!  Arbeitsjurist
!!! new !!  Kristallograph
!!! new !!  Fitnessökonom
!!! new !!  Ethnograph
!!! new !!  Sozialgerontologe
!!! new !!  Technik-Übersetzer
!!! new !!  Buchgestalter
!!! new !!  Fotografiker
!!! new !!  Sozialpädagoge
!!! new !!  Vietnamist
!!! new !!  Kulturarbeiter
!!! new !!  Mongolist
!!! new !!  Fertigungsprozessinformatiker
!!! new !!  Wirtschaftshispanist
!!! new !!  Film- und Fernsehdramaturg
!!! new !!  Agrarpädagoge
!!! new !!  Gartenbaupädagoge
168
29


# Check all occupations with "-" and " "

In [7]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
a=[]    
def change_all_with_minus():
    berufe=load_simple_json("de/berufe.json")
    to_del=[]
    new_beruf={}
    #find all "paedago" and change them to "paedagoge"
    for beruf in berufe:
        if " f\xfcr ".decode('ISO 8859-16') in beruf:
            print beruf
            a.append(beruf)
        if beruf[-4:]=="dago":
            new_beruf[beruf+"ge"]=berufe[beruf]
            to_del.append(beruf)
            print "We will change mistake:",beruf
        if "Braunmeister" in beruf:
            new_beruf[beruf.replace("Braunmeister","Braumeister")]=berufe[beruf].replace("Braunmeisterin","Braumeisterin")
            to_del.append(beruf)

    #del old version
    for i_i in to_del:
        del berufe[i_i]
    #add new version
    for b in new_beruf:
        berufe[b]=new_beruf[b]

    #look into all occupations with "und"&"-" 	
    fremd_words=["Avid","Finno","Corporate","General","Media","Account","Media","Management","Pre-Opening"]
    i=0
    k=0
    add_new={}
    del_old=[]
    very_rare=load_simple_json("de/very_rare.json")
    for beruf in berufe:
        if ("-" in beruf) | ("- u" in beruf) | ("-, " in beruf)| (". " in beruf)| (" - " in beruf)| ("- " in beruf) | ("," in beruf):
            if ("," in beruf)&("-" not in beruf):
                if "/" in beruf:
                    #print beruf
                    new_b1=beruf.split(",")[0].strip()
                    new_b2=beruf.split(",")[1].strip()
                    if "/" not in new_b1:
                        if not berufe.has_key(new_b1):
                            #print "We should add ",new_b1
                            add_new[new_b1]=berufe[beruf].split(",")[0]
                            #print add_new
                        #new_b2 with /
                        new_b2=new_b2.split("/")
                        if not berufe.has_key(new_b2[0]):
                            add_new[new_b2[0]]=new_b2[0]+"in" #VERY naive!!!
                            #print add_new
                    else:
                        if not berufe.has_key(new_b2):
                            #print "We should add ",new_b2
                            add_new[new_b2]=berufe[beruf].split(",")[1]
                            #print add_new
                        #new_b1 with /
                        new_b1=new_b1.split("/")
                        if not berufe.has_key(new_b1[0]):
                            add_new[new_b1[0]]=new_b1[0]+"in" #VERY naive!!!
                            #print add_new
                    del_old.append(beruf)
            else:
                #print beruf
                if "-" in beruf:
                    if (" " not in beruf) & ("," not in beruf):
                        #here we have 182 beruf, e.g. "NC-Anwendungsfachmann" and we will do nothing with them
                        #print beruf
                        #k+=1
                        pass
                    else:
                        new_=beruf.split("-")
                        if new_[1] not in fremd_words:
                            if ("REFA" in beruf)|("Pre-Opening" in beruf):#exclusion:
                                pass
                            elif ("-techn. " in beruf):
                                del_old.append(beruf)
                                add_new[beruf.replace("-techn.","technischer")]=berufe[beruf].replace("-techn.","technische")
                            #"Veterinärmedizinisch-techn. Lehrassistent"
                            #"Neurophysiologisch-techn. Assistent","Pre-Opening Manager","REFA - Prozessengineer"
                            elif ("Atem-" in beruf)&("Sprech-" in beruf):
                                del_old.append(beruf)
                                add_new["Atem-, Sprech- und Stimmlehrer"]="Atem-, Sprech- und Stimmlehrerin"
                                #Lehrkraft - Atem-, Sprech-, Stimmlehrer
                                #Atem-,Sprech-,Stimmlehrer
                            elif "Lehrkraft - Fachschulen - Dorfhelfer" in beruf:
                                del_old.append(beruf)  
                            elif "Wirtschaftsinformatik" in beruf:
                            #Assistenten und Assistentinnen für Wirtschaftsinformatik
                                del_old.append(beruf)
                                add_new["Assistent für Wirtschaftsinformatik".decode('utf-8')]="Assistentin für Wirtschaftsinformatik".decode('utf-8')
                            
                            elif "Kfz - " in beruf:
                                del_old.append(beruf)
                                add_new[beruf.replace(" - ","-")]=berufe[beruf].replace(" - ","-")
                            elif ", -" in beruf:
                                #!!!we leave two berufe
                                #"Keramgravierer, -\u00e4tzer"
                                #"Konservierungsguttrockner, -erhitzer"
                                del_old.append(beruf)
                                add_new[beruf]=berufe[beruf].replace("er,","erin,")
                            elif " u. " in beruf:
                                #!!!we leave these berufe together
                                del_old.append(beruf)
                                add_new[beruf.replace(" u. "," und ")]=berufe[beruf].replace(" u. "," und ") 
                            elif " und " in beruf:
                                #!!!we leave these berufe together
                                pass
                            else:
                                #print beruf #here we have 
                                k+=1
                                beruf_=beruf.replace(", ",",").replace(",",", ")
                                beruf_f=berufe[beruf].replace(", ",",").replace(",",", ")
                                if "-, " in beruf_:
                                    del_old.append(beruf)
                                    add_new[beruf_.replace("-, ", "- und ")]=berufe[beruf].replace(", ",",").replace(",",", ").replace("-, ", "- und ")
                                elif"," in beruf_:
                                    del_old.append(beruf)
                                    if beruf_.count(',')==1:
                                        beruf_=beruf_.replace(","," und")
                                        beruf_f=beruf_f.replace(","," und")
                                    else:
                                        beruf_=beruf_.replace(","," und").replace(" und",",",1)
                                        beruf_f=beruf_f.replace(","," und").replace(" und",",",1)
                                    add_new[beruf_]=beruf_f
                                    
                else:
                    #print beruf
                    very_rare.update({beruf:berufe[beruf]})
                    del_old.append(beruf)
                
            i+=1
        elif " " in beruf:
            splitted=beruf.split(" ",1)
            if splitted[0] not in fremd_words:
                pass
                #print beruf
                #here "Nautischer Schiffsoff." should be neutrale form ---> check it
                #!!!Still not done! Do not forget
            if "  " in beruf:
                add_new[beruf.replace("  "," ")]=berufe[beruf].replace("  "," ")
                beruf.replace("  "," ")
                del_old.append(beruf)
                #print "with two spaces:"beruf
        elif "." in beruf:
            print beruf
            del_old.append(beruf)
        if (" und " in beruf)&("-" not in beruf):
            #print beruf
            new_b1=beruf.split(" und ")
            if not berufe.has_key(new_b1[0]):
                #print "add!!"
                add_new[new_b1[0]]=berufe[beruf].split(" und ")[0]
            if not berufe.has_key(new_b1[1]):
                #print "addd!!!"
                add_new[new_b1[1]]=berufe[beruf].split(" und ")[1]
        elif (" u. " in beruf)&("-" not in beruf):
            print beruf
    print i,k #578

    #del old version
    for i_i in del_old:
        del berufe[i_i]
    #add new version
    for b in add_new:
        berufe[b]=add_new[b]
    with open('de/very_rare.json', 'w') as f:
        json.dump(very_rare,  f, encoding='ISO 8859-16', indent=4)
        
    with open('de/berufe.json', 'w') as f:
        json.dump(berufe,  f, encoding='ISO 8859-16', indent=4) 
change_all_with_minus()

Verantwortlicher für Veranstaltungstechnik
We will change mistake: Religionspädago
Naut.Schiffsoffizier
Dipl.Agraringenieur
558 135


# Check file "neutral.json"

In [8]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)


words=load_simple_json('de/neutral.json')
print len(words)
neutral=[]
words_all=load_simple_json('de/occupation_all.json')
words_revert={words_all[i][1]:i for i in words_all}#feminine:masculine
for i in words:
    if i not in words_all:
        if i not in words_revert:
            neutral.append(i)
        else:
            print "==F:",i
    else:
        print "===M:",i
        
print len(neutral)  

with open('de/neutral_new.json', 'w') as f:
    json.dump(neutral,  f,  indent=4)

299
===M: Fotodesigner
===M: Design-Ingenieur
===M: Schmuckdesigner
===M: Feinwerktechniker
===M: Schmuckgestalter
===M: Edelsteingestalter
===M: Maskenbildner
===M: Sozialgerontologe
===M: Altbauerneuerungstechniker
===M: Bewährungshelfer
===M: Sozialjurist
===M: Bauvermessungstechniker
===M: Landbautechniker
===M: Textildesigner
===M: Sozialpädagoge
===M: Bergbautechniker
===M: Hüttentechniker
===M: Baudenkmalpfleger
===M: Korrosionsschutztechniker
==F: Ingenieurin
===M: Abwassertechniker
===M: Elektrotechniker
277


# Parse file "all_with_fur.json"

In [9]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
######### file  de/occupation_all.json should be produced before. otherwise use file berufe!!!!!   
berufe=load_simple_json("de/occupation_all.json")
berufe_revert={berufe[i][1]:i for i in berufe}#feminine:masculine  
all_with_fur_neutral=load_simple_json("de/all_with_fur_neutral.json")
all_with_fur=load_simple_json("de/all_with_fur.json")
neutral_beruf=load_simple_json("de/neutral_new.json")
all_with_fur_berufe=load_simple_json("de/all_with_fur_berufe.json")

#neutral forms
for i in  all_with_fur_neutral:
    beruf_=i.split(" f\xfcr ".decode('ISO 8859-16'))
    b_with_fur=beruf_[0].split('/')        
    if (beruf_[0] in berufe):
        print "We already have: ",beruf_[0]
        #do smth 
    elif (beruf_[0] in berufe_revert):
        print "We already have: ",beruf_[0]
    else:  
        if i not in neutral_beruf:
            neutral_beruf.append(i)
            
            
i=0        
for beruf in  all_with_fur:
    if " f\xfcr ".decode('ISO 8859-16') in beruf:
        beruf=beruf.replace(" u."," und").replace(".u.",". und ").strip()
        if "." in beruf:
            beruf=check_dot(beruf)
        if "-/" in beruf:
            beruf=beruf.replace("-/","-, ")
        beruf_=beruf.split(" f\xfcr ".decode('ISO 8859-16'))
        if "/" in beruf_[1]:
            beruf_[1]=beruf_[1].replace("/"," und ")
        b_with_fur=beruf_[0].split('/')
        if len(b_with_fur)==2:
            i+=1    
            if "frau" in b_with_fur[1]:
                all_with_fur_m=b_with_fur[0]+" f\xfcr ".decode('ISO 8859-16')+beruf_[1]
                all_with_fur_f=b_with_fur[0][:-4]+"frau f\xfcr ".decode('ISO 8859-16')+beruf_[1]                
            elif b_with_fur[1][0]=="r":
                all_with_fur_m=beruf.replace("/r","r")
                all_with_fur_f=beruf.replace("/r","")
            elif b_with_fur[1][:2]=="in":
                all_with_fur_m=beruf.replace("/in","")
                all_with_fur_f=beruf.replace("/in","in")
            else:
                pass
                #print beruf    
            if "/" in all_with_fur_m:
                all_with_fur_m=all_with_fur_m.replace("/"," und ")
                all_with_fur_f=all_with_fur_f.replace("/"," und ")
            all_with_fur_berufe[all_with_fur_m]=all_with_fur_f

berufe=load_simple_json("de/berufe.json")
berufe.update(all_with_fur_berufe)
del_old=[]
add_new={}
for j in all_with_fur_berufe:
    if " - " in j:
        new_beruf_=j.split(" - ")
        #e.g. Facharbeiter für Fertigungsmittel - Schneidewerkzeuge ===> try ohne "- Schneidewerkzeuge"
        if new_beruf_[0] not  in berufe.keys():
            berufe[new_beruf_[0]]=all_with_fur_berufe[j].split(" - ")[0]  
            print new_beruf_[0]

#Now we will del all  " - "           
for j in berufe:
    if " - " in j:
        del_old.append(j)
        add_new[j.replace(" - "," im ")]=berufe[j].replace(" - "," im ")
    elif ("-, " in j)&(" und " not in j):
        print j
        del_old.append(j)
        add_new[j.replace("-, ","- und  ")]=berufe[j].replace("-, ","- und  ")
#del old version
for i_i in del_old:
    del berufe[i_i]
#add new version
for b in add_new:
    berufe[b]=add_new[b]        
        
print i    
with open('de/neutral_new.json', 'w') as f:
    json.dump(neutral_beruf,  f, encoding='ISO 8859-16', indent=4)   
with open('de/all_with_fur_berufe.json', 'w') as f:
    json.dump(all_with_fur_berufe,  f, encoding='ISO 8859-16', indent=4)  
with open('de/berufe.json', 'w') as f:
    json.dump(berufe,  f, encoding='ISO 8859-16', indent=4) 

We already have:  Ingenieurin
Facharbeiter für Werkzeugmaschinen
Kaufmann für Verkehrsservice
Facharbeiter für Anlagentechnik
Facharbeiter für Fertigungsmittel
Facharbeiter für Nachrichtentechnik
Fachpraktiker für Kreislauf-, Abfallwirtschaft
Fachpraktiker für Land-, Baumaschinentechnik
73


In [10]:
print len(berufe)

4163


In [35]:
neutral_new_revised=load_simple_json("de/neutral_new_revised.json")
neutral_word=[]
new_word={}
for i in neutral_new_revised:     
    if neutral_new_revised[i]==1:
        neutral_word.append(i.replace(" u. "," und "))
    else:
        new_word[i.replace(" u. "," und ").replace(", -"," und -")]=neutral_new_revised[i].replace(" u. "," und ").replace(", -"," und -")
print "We have",len(neutral_word),"neutral words and ",len(new_word),"new words." 


with open('de/neutral_cleaned.json', 'w') as f:
    json.dump(neutral_word,  f, encoding='ISO 8859-16', indent=4) 

#add to berufe
berufe=load_simple_json('de/berufe.json') 

for i in new_word:
    if i not in berufe:
        berufe[i]=new_word[i]
    else:
        print "We already have it in our list: ",i
    
print "Now file berufe consist of ",len(berufe),"words."   

with open('de/berufe.json', 'w') as f:
    json.dump(berufe,  f, encoding='ISO 8859-16', indent=4)     

We have 189 neutral words and  79 new words.
We already have it in our list:  Diskjockey
We already have it in our list:  Feldwebel - Sanitätsdienst
We already have it in our list:  Innanarchitekt
We already have it in our list:  Wirkereitechniker
We already have it in our list:  Gestaltungsdirektor
We already have it in our list:  Bauwerkerhaltungstechniker
We already have it in our list:  Matrose - Binnenschifffahrt
We already have it in our list:  Fachunteroffizier - Allgemeiner Fachdienst
We already have it in our list:  Recruiter
We already have it in our list:  Katholischer Pfarrer
We already have it in our list:  Brautechniker
We already have it in our list:  Diakon
We already have it in our list:  Schiffsjunge
We already have it in our list:  Qualitätstechniker
We already have it in our list:  Feinkeramtechniker
We already have it in our list:  Blechbearbeitungstechniker
We already have it in our list:  Verpackungstechniker
We already have it in our list:  Entsorgungstechniker


# Match two files

In [36]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import xml.dom.minidom
import json

word={}

#Open File
dateiXML = open("de/Worttabelle.xml", "r")
dateiJSON = open("de/berufe.json", "r")


#Parse File
parsedXMLFile = xml.dom.minidom.parse(dateiXML)
parseJSONFile = json.load(dateiJSON)

#Close File
dateiXML.close()
dateiJSON.close()


def liesText(knoten):
    for k in knoten.childNodes:
        if k.nodeType == k.TEXT_NODE:
            return k.nodeValue.strip()
 
def liesWorttabelle(knoten):
    for num, elem in enumerate(knoten.getElementsByTagName("Worttabelle")):
        
        for knotenHauptwort in elem.getElementsByTagName("Hauptwort"):
            hauptwort = liesText(knotenHauptwort)
            
        for knotenVorschlag in elem.getElementsByTagName("Vorschlag"):
            vorschlag = liesText(knotenVorschlag)
            
        if word.has_key(hauptwort):
            word[hauptwort].append(vorschlag)
        else:
            word[hauptwort]=[vorschlag]
 
def dokument(start):
    for elem in start.getElementsByTagName("dataroot"):
        liesWorttabelle(elem)
 
dokument(parsedXMLFile)
#print word
#print parseJSONFile

word_neutral={}
for i in word:
    if i in parseJSONFile.keys():
        word_neutral[i]=[word[i],parseJSONFile[i]]



with open("de/occupation.json","w")as f:
    json.dump(word_neutral, f, indent=4)


print len(word_neutral)
print len(word)

82
682


In [37]:
def upcase_first_letter(s):
    return s[0].upper() + s[1:]

other_prof_sub={}

for i in parseJSONFile:
    if i in word.keys():
        pass
        #word_neutral[i]=[word[i],parseJSONFile[i]] #masculine:[[neutrale],[feminine]]
    else:
        for prof_m in word:
            if i in prof_m:
                if (i=="Zahnarzt")|(i=="Arzt"):    #pass if Arzt                    
                    pass
                elif not word_neutral.has_key(i):
                    other_prof_sub[i]=[word[prof_m],parseJSONFile[i]]
                    #print i , word[prof_m]
                else:
                    pass
            elif i.lower() in prof_m:
                if other_prof_sub.has_key(i):
                    pass
                    
                else:
                    print i, prof_m
                    befor,meadle,after=prof_m.partition(i.lower())
                    if (meadle==i.lower() ) & (after=="") & ("Fr. " not in prof_m )&("Frau " not in prof_m):#only if difference in first part
                        temp=[]
                        for x in word[prof_m]:
                            if " und " in x:
                                temp.append(x.replace(befor,"").title().replace(" Und ", " und "))
                            else:
                                temp.append(upcase_first_letter(x.replace(befor,"")))
                        other_prof_sub[i]=[temp,parseJSONFile[i]]
                        print "Ready:",i,other_prof_sub[i]
                    
print len(word_neutral)                
print len(other_prof_sub)
new_prof={}
new_prof=other_prof_sub.copy()
new_prof.update(word_neutral)
print len(new_prof)


with open("de/occupation_sub.json","w")as f:
    json.dump(new_prof, f, indent=4)

Meister Bademeistern
Meister Bürgermeister
Ready: Meister [[u'Meisterinnen und Meister', u'MeisterInnen'], u'Meisterin']
Installateur Elektroinstallateure
Inspektor Fr. Fachinspektor
Inspektor Frau Fachinspektor
Inspektor Frau Fachoberinspektor
Inspektor Fr. Fachoberinspektor
Bearbeiter Sachbearbeiter
Ready: Bearbeiter [[u'Bearbeiterinnen und Bearbeiter', u'BearbeiterInnen'], u'Bearbeiterin']
Offizier Unteroffizieren
Offizier Unteroffiziere
Ingenieur Bauingenieure
Ingenieur Maschinenbauingenieure
Ingenieur Bauingenieuren
Ingenieur Elektroingenieure
Helfer Zahnarzthelfer
Ready: Helfer [[u'Helferinnen und Helfer', u'HelferInnen'], u'Helferin']
Hauer Bildhauern
Hauer Fleischhauer
Ready: Hauer [[u'Hauerinnen und Hauer', u'HauerInnen'], u'Hauerin']
Roller Controller
Ready: Roller [[u'RollerInnen', u'Rollerinnen und Roller'], u'Rollerin']
Kurator Fr. Generalprokurator
Kurator Frau Generalprokurator
Praktikant Ferialpraktikanten
Assistent Anwaltsassistenten
Assistent Universitätsassistenten
S

In [38]:
word_neutral_all={}

for i in parseJSONFile:
    if i in new_prof.keys():
        if parseJSONFile[i]+" oder "+i not in new_prof[i][0]:
            n,f=new_prof[i]
            n.append(parseJSONFile[i]+" oder "+i)
            new_prof[i]=[n,f]
        if parseJSONFile[i]+" und "+i not in new_prof[i][0]:
            n,f=new_prof[i]
            n.append(parseJSONFile[i]+" und "+i)
            new_prof[i]=[n,f]
        if parseJSONFile[i]+" bzw. "+i not in new_prof[i][0]:
            n,f=new_prof[i]
            n.append(parseJSONFile[i]+" bzw. "+i)
            new_prof[i]=[n,f]
    else:
        #via slesh Fachmann/Fachfrau
        #via DekateurIn=Dekateur/in
        #lerer lehrerin lehrperson!!!lehrkraft!! (still not done) 
        #mann #frau# ->leute, kraft , ?person 
        befor,meadle,after=i.partition(parseJSONFile[i])
        if (befor=="")&(meadle==parseJSONFile[i])&(after=="r"):
            #print i, parseJSONFile[i]
            word_neutral_all[i]=[[i[:-1]+"R",parseJSONFile[i]+" und "+i,parseJSONFile[i]+" oder "+i,
                                 parseJSONFile[i]+" bzw. "+i],parseJSONFile[i]]
        elif parseJSONFile[i]==i+"in":
            #print i, parseJSONFile[i]
            #elif (parseJSONFile[i][-2:]=="in")&(i[-2:]=="er"):
            word_neutral_all[i]=[[i+"In",parseJSONFile[i]+" und "+i,parseJSONFile[i]+" oder "+i,
                                 parseJSONFile[i]+" bzw. "+i],parseJSONFile[i]]
        elif (parseJSONFile[i][-4:]=="frau")&(i[-4:]=="mann"): 
            word_neutral_all[i]=[[i[:-4]+"kraft",i[:-4]+"leute",parseJSONFile[i]+" und "+i,parseJSONFile[i]+" oder "+i,
                                 parseJSONFile[i]+" bzw. "+i],parseJSONFile[i]]
        else:
            word_neutral_all[i]=[[parseJSONFile[i]+" und "+i, parseJSONFile[i]+" oder "+i,
                                 parseJSONFile[i]+" bzw. "+i],parseJSONFile[i]]

word_neutral_all_copy=word_neutral_all.copy()
word_neutral_all_copy.update(new_prof)
print len(word_neutral_all_copy)
with open("de/occupation_all.json","w")as f:
    json.dump(word_neutral_all_copy, f, indent=4)


4240


# cleaning file manually

In [1]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
cleaned=load_simple_json("de/occupation_all.json")  

del cleaned["Krankenschwester"]

with open("de/occupation_all.json","w")as f:
    json.dump(cleaned, f, indent=4)